# Climate-Driven Disease Outbreak Prediction 

## Importing Libraries

In [1]:
import pandas as pd
import matplotlib.pyplot as plt

## Load The Dataset

In [3]:
df = pd.read_csv("../Data/processed/India_Climate_Disease_Merged_Monthly.csv")

## Explore The Dataset

In [4]:
# Check first few rows
print("Preview of dataset:")
display(df.head())

Preview of dataset:


,Year,Month,TempAnomaly,DengueCases,MalariaIncidence,Humidity,Precipitation
0,2000,1,0.832,650.0,19.956696,56.250000,10.38
1,2000,2,-0.670,650.0,19.956696,53.006667,17.44
2,2000,3,-0.412,650.0,19.956696,49.763333,12.11
3,2000,4,1.127,650.0,19.956696,46.520000,33.27
4,2000,5,0.040,650.0,19.956696,55.133333,72.38


In [5]:
# Info about columns and datatypes
print("\nDataset Info:")
print(df.info())



Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 252 entries, 0 to 251
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Year              252 non-null    int64  
 1   Month             252 non-null    int64  
 2   TempAnomaly       252 non-null    float64
 3   DengueCases       252 non-null    float64
 4   MalariaIncidence  252 non-null    float64
 5   Humidity          247 non-null    float64
 6   Precipitation     252 non-null    float64
dtypes: float64(5), int64(2)
memory usage: 13.9 KB
None


In [6]:
# Descriptive statistics
print("\nDescriptive Statistics:")
display(df.describe())


Descriptive Statistics:


,Year,Month,TempAnomaly,DengueCases,MalariaIncidence,Humidity,Precipitation
count,252.000000,252.000000,252.000000,252.000000,252.000000,247.000000,252.000000
mean,2010.000000,6.500000,0.607798,46170.285714,13.435015,61.548745,92.965833
std,6.067351,3.458922,0.576033,54312.172454,6.589167,8.385674,98.629710
min,2000.000000,1.000000,-1.146000,650.000000,2.653887,45.640000,3.340000
25%,2005.000000,3.750000,0.268500,11985.000000,6.700888,55.130000,15.815000
50%,2010.000000,6.500000,0.590500,18860.000000,17.323007,63.520000,46.270000
75%,2015.000000,9.250000,0.920250,55153.000000,18.464534,69.035000,161.835000
max,2020.000000,12.000000,2.540000,188401.000000,22.480590,75.190000,341.370000


In [7]:
# Missing values check
print("\nMissing Values per Column:")
print(df.isnull().sum())


Missing Values per Column:
Year                0
Month               0
TempAnomaly         0
DengueCases         0
MalariaIncidence    0
Humidity            5
Precipitation       0
dtype: int64
